In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.9 MB/s eta 0:00:00


In [3]:
!pip install deeppavlov

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.7/433.7 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.2/145.2 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 KB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 KB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [13]:
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import pickle
import numpy as np
import os
import time
from stuff import generate_text, respond_to_dialog
from deeppavlov import build_model, configs
import transformers

In [34]:
tokenizer_classif = AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_classif(sentence):
    tokens = tokenizer_classif.encode_plus(sentence, max_length=30,
                                   truncation=True, padding='max_length',
                                   add_special_tokens=True, return_attention_mask=True,
                                   return_token_type_ids=False, return_tensors='tf')
    return tokens['input_ids'], tokens['attention_mask']

In [28]:
first_classif_model = tf.keras.models.load_model('models/first_classif.h5', custom_objects={"TFBertModel": transformers.TFBertModel})

In [27]:
device = torch.device("cuda")

In [26]:
tokenizer_boltalka = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
model_boltalka = AutoModelForCausalLM.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2').to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
model_stihov = tf.keras.models.load_model('models/generator_stihov.h5')

In [20]:

model_context = build_model(configs.squad.squad_ru_bert, download=True)
context = open('sh.txt', 'rb').read().decode(encoding='utf-8', errors="ignore")

2023-03-14 14:00:51.741 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/squad/squad_ru_torch_bert.tar.gz download because of matching hashes
INFO:deeppavlov.download:Skipped http://files.deeppavlov.ai/v1/squad/squad_ru_torch_bert.tar.gz download because of matching hashes
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraini

In [37]:
test = 'Где был подрыв?'

In [42]:
model_context([context], [test])[0][0]

'в Панама-Сити'

In [47]:
def answer(mytext):
    Xids, Xmask = tokenize_classif(mytext)
    classif_text = {'input_ids': Xids, 'attention_mask': Xmask}
    result = first_classif_model(classif_text)
    numclass = np.argmax(result[0])
    prob = np.max(result[0])
    
    try:
        if numclass == 0 and prob > 0.8:
            answ = generate_text(model_stihov, mytext)
        elif numclass == 1 and prob > 0.8:
            answ = model_context([context], [mytext])[0][0]
        else:
            answ = respond_to_dialog(model_boltalka, tokenizer_boltalka, [mytext])

    except:
        answ = respond_to_dialog(model_boltalka, tokenizer_boltalka, [mytext])
    print(answ)

In [52]:
answer('Что сказал Байден?')

Байден этого попросту не понял или проигнорировал


In [50]:
answer('Продолжи стих: Я вас любил')

Продолжи стих: Я вас любил! таков и ныне я:
		Беспечный, влюбчивый. Вы встретлов он.
		На серебро, на злачное супруты,
		Обнимет вас, и тихою рукою
		На мягкой молоденечи под отзорстие неподогда, гонят от него рукой,
		Пр


In [51]:
answer('Кто готовил водолазов для подрыва?')

из Панама-Сити


In [55]:
answer('Ты любишь кушать мороженное?')

Я люблю мороженое.


In [56]:
answer('Придумай продолжение: Буря небо мглою кроет')

Придумай продолжение: Буря небо мглою кроет покрывала,
		И слабо в бледных облаках
		Увидел их Махров Головку прекрасных мечтаний
		С коленом пеною поразил
		Во мне надежду, скорбь и радость
		[И душу] [бурную]
		[Готова холодна?
		Ужел
